In [ ]:
import warnings
warnings.filterwarnings("ignore")

import ast
import pickle
import IPython.display as ipd
import librosa
from matplotlib import pyplot as plt
import numpy as np
import os
import time
import pandas as pd
from collections import Counter
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from collections import Counter, OrderedDict
import cv2
from google.colab.patches import cv2_imshow
import random
import argparse
import sys
import math
from tqdm.notebook import tqdm as tq
import seaborn as sns
import xml.etree.ElementTree as ET

import joblib

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, SVR
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier, SGDRegressor
from sklearn import metrics
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.exceptions import NotFittedError
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.utils import class_weight
from sklearn.utils.class_weight import compute_class_weight

import keras
from keras.models import Sequential, load_model, Model
from keras.layers import Input, Dense, Dropout, Flatten, Conv2D, Conv1D, Conv3D, MaxPooling3D, MaxPooling2D, MaxPooling1D, LeakyReLU, Conv2DTranspose, Concatenate, concatenate, add
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras import backend as K
from keras.utils import to_categorical
!pip install keras_applications
from keras import backend
from keras.applications import imagenet_utils
from keras_applications.imagenet_utils import _obtain_input_shape
from keras.applications import VGG16, VGG19

import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop,Adamax, Nadam
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image_dataset_from_directory

try:
    from tensorflow.python.util import module_wrapper as deprecation
except ImportError:
    from tensorflow.python.util import deprecation_wrapper as deprecation
deprecation._PER_MODULE_WARNING_LIMIT = 0
from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
comm_detail = pd.read_csv('/content/drive/MyDrive/merged_file_1.csv')
comm_detail

,Arrival_Date,State,District,Market,Commodity,Variety,Grade,Min_Price,Max_Price,Modal_Price,Commodity_Code
0,2001-01-10,Karnataka,Dharwad,Hubli (Amaragol),Black Gram (Urd Beans)(Whole),Black Gram (Whole),FAQ,1380.0,1380.0,1380.0,8
1,2001-01-10,Karnataka,Dharwad,Hubli (Amaragol),Green Gram (Moong)(Whole),Green (Whole),FAQ,1600.0,1900.0,1800.0,9
2,2001-02-08,Maharashtra,Kolhapur,Kolhapur,Brinjal,Round,FAQ,500.0,1060.0,775.0,35
3,2001-02-08,Maharashtra,Kolhapur,Kolhapur,Guar,Other,FAQ,1400.0,1700.0,1550.0,75
4,2001-03-12,Nagaland,Kohima,Khuzama,Potato,Other,FAQ,500.0,700.0,600.0,24
...,...,...,...,...,...,...,...,...,...,...,...
9283995,2023-12-31,Gujarat,Vadodara(Baroda),Vadodara(Sayajipura),Ginger(Green),Other,FAQ,8000.0,9000.0,8550.0,103
9283996,2023-12-31,Karnataka,Shimoga,Shimogga(Theertahalli),Arecanut(Betelnut/Supari),Gorabalu,FAQ,30870.0,37209.0,36501.0,140
9283997,2023-12-31,Maharashtra,Ahmednagar,Rahata,Ginger(Green),Other,FAQ,3500.0,10000.0,6500.0,103
9283998,2023-12-31,Rajasthan,Jodhpur,Jodhpur (Grain),Mustard,Other,FAQ,4300.0,4600.0,4450.0,12


In [ ]:
comm_detail = comm_detail.dropna()
comm_detail

,Arrival_Date,State,District,Market,Commodity,Variety,Grade,Min_Price,Max_Price,Modal_Price,Commodity_Code
0,2001-01-10,Karnataka,Dharwad,Hubli (Amaragol),Black Gram (Urd Beans)(Whole),Black Gram (Whole),FAQ,1380.0,1380.0,1380.0,8
1,2001-01-10,Karnataka,Dharwad,Hubli (Amaragol),Green Gram (Moong)(Whole),Green (Whole),FAQ,1600.0,1900.0,1800.0,9
2,2001-02-08,Maharashtra,Kolhapur,Kolhapur,Brinjal,Round,FAQ,500.0,1060.0,775.0,35
3,2001-02-08,Maharashtra,Kolhapur,Kolhapur,Guar,Other,FAQ,1400.0,1700.0,1550.0,75
4,2001-03-12,Nagaland,Kohima,Khuzama,Potato,Other,FAQ,500.0,700.0,600.0,24
...,...,...,...,...,...,...,...,...,...,...,...
9283995,2023-12-31,Gujarat,Vadodara(Baroda),Vadodara(Sayajipura),Ginger(Green),Other,FAQ,8000.0,9000.0,8550.0,103
9283996,2023-12-31,Karnataka,Shimoga,Shimogga(Theertahalli),Arecanut(Betelnut/Supari),Gorabalu,FAQ,30870.0,37209.0,36501.0,140
9283997,2023-12-31,Maharashtra,Ahmednagar,Rahata,Ginger(Green),Other,FAQ,3500.0,10000.0,6500.0,103
9283998,2023-12-31,Rajasthan,Jodhpur,Jodhpur (Grain),Mustard,Other,FAQ,4300.0,4600.0,4450.0,12


In [ ]:
for i in comm_detail.columns:
  print(i, comm_detail[i].dtype)

Arrival_Date object
State object
District object
Market object
Commodity object
Variety object
Grade object
Min_Price float64
Max_Price float64
Modal_Price float64
Commodity_Code int64


In [ ]:
comm_detail['Arrival_Date'] = pd.to_datetime(comm_detail['Arrival_Date'])
comm_detail['Arrival_Date'].dtype

dtype('<M8[ns]')

In [ ]:
comm_detail['Year'] = comm_detail['Arrival_Date'].dt.year
comm_detail['Month'] = comm_detail['Arrival_Date'].dt.month
comm_detail['Day'] = comm_detail['Arrival_Date'].dt.day
comm_detail

,Arrival_Date,State,District,Market,Commodity,Variety,Grade,Min_Price,Max_Price,Modal_Price,Commodity_Code,Year,Month,Day
0,2001-01-10,Karnataka,Dharwad,Hubli (Amaragol),Black Gram (Urd Beans)(Whole),Black Gram (Whole),FAQ,1380.0,1380.0,1380.0,8,2001,1,10
1,2001-01-10,Karnataka,Dharwad,Hubli (Amaragol),Green Gram (Moong)(Whole),Green (Whole),FAQ,1600.0,1900.0,1800.0,9,2001,1,10
2,2001-02-08,Maharashtra,Kolhapur,Kolhapur,Brinjal,Round,FAQ,500.0,1060.0,775.0,35,2001,2,8
3,2001-02-08,Maharashtra,Kolhapur,Kolhapur,Guar,Other,FAQ,1400.0,1700.0,1550.0,75,2001,2,8
4,2001-03-12,Nagaland,Kohima,Khuzama,Potato,Other,FAQ,500.0,700.0,600.0,24,2001,3,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9283995,2023-12-31,Gujarat,Vadodara(Baroda),Vadodara(Sayajipura),Ginger(Green),Other,FAQ,8000.0,9000.0,8550.0,103,2023,12,31
9283996,2023-12-31,Karnataka,Shimoga,Shimogga(Theertahalli),Arecanut(Betelnut/Supari),Gorabalu,FAQ,30870.0,37209.0,36501.0,140,2023,12,31
9283997,2023-12-31,Maharashtra,Ahmednagar,Rahata,Ginger(Green),Other,FAQ,3500.0,10000.0,6500.0,103,2023,12,31
9283998,2023-12-31,Rajasthan,Jodhpur,Jodhpur (Grain),Mustard,Other,FAQ,4300.0,4600.0,4450.0,12,2023,12,31


In [ ]:
cols = ['Commodity','State','District','Market','Variety','Grade']
for col in cols:
  unique_val = comm_detail[col].unique()
  print(col)
  sort_val = sorted(unique_val)
  for ind, i in enumerate(sort_val):
    print(ind, i)

Streaming output truncated to the last 5000 lines.
107 Chhapra
108 Chhatarpur
109 Chhindwara
110 Chhota Udaipur
111 Chikmagalur
112 Chitradurga
113 Chitrakut
114 Chittor
115 Chittorgarh
116 Churu
117 Coimbatore
118 Coochbehar
119 Cuddalore
120 Cuddapah
121 Cuttack
122 Dahod
123 Dakshin Dinajpur
124 Damoh
125 Dantewada
126 Darbhanga
127 Darjeeling
128 Darrang
129 Datia
130 Dausa
131 Davangere
132 Dehradoon
133 Delhi
134 Deogarh
135 Deoria
136 Devbhumi Dwarka
137 Dewas
138 Dhalai
139 Dhamtari
140 Dhanbad
141 Dhar
142 Dharashiv(Usmanabad)
143 Dharmapuri
144 Dharwad
145 Dhemaji
146 Dhenkanal
147 Dholpur
148 Dhubri
149 Dhule
150 Dibrugarh
151 Dimapur
152 Dindigul
153 Dindori
154 Dumka
155 Dungarpur
156 Durg
157 East
158 East Champaran/ Motihari
159 East Garo Hills
160 East Godavari
161 East Jaintia Hills
162 East Khasi Hills
163 East Siang
164 East Singhbhum
165 Ernakulam
166 Erode
167 Etah
168 Etawah
169 Faridabad
170 Faridkot
171 Farukhabad
172 Fatehabad
173 Fatehgarh
174 Fatehpur
175 Faz

In [ ]:
le = LabelEncoder()
label_encodings = {}  # Dictionary to store all label encodings
for col in cols:
  print(col)
  comm_detail[col] = le.fit_transform(comm_detail[col])
  le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
  print(le_name_mapping)
  print("################")

joblib.dump(label_encodings, '/content/drive/MyDrive/models/label_encoders.pkl')

print("Label encodings saved to label_encoders.pkl")

Commodity
{'Ajwan': np.int64(0), 'Alasande Gram': np.int64(1), 'Almond(Badam)': np.int64(2), 'Alsandikai': np.int64(3), 'Amaranthus': np.int64(4), 'Ambada Seed': np.int64(5), 'Amla(Nelli Kai)': np.int64(6), 'Amphophalus': np.int64(7), 'Antawala': np.int64(8), 'Anthorium': np.int64(9), 'Apple': np.int64(10), 'Apricot(Jardalu/Khumani)': np.int64(11), 'Arecanut(Betelnut/Supari)': np.int64(12), 'Arhar (Tur/Red Gram)(Whole)': np.int64(13), 'Arhar Dal(Tur Dal)': np.int64(14), 'Ashgourd': np.int64(15), 'Astera': np.int64(16), 'Avare Dal': np.int64(17), 'BOP': np.int64(18), 'Bajra(Pearl Millet/Cumbu)': np.int64(19), 'Balekai': np.int64(20), 'Bamboo': np.int64(21), 'Banana': np.int64(22), 'Banana - Green': np.int64(23), 'Barley (Jau)': np.int64(24), 'Bay leaf (Tejpatta)': np.int64(25), 'Beans': np.int64(26), 'Beaten Rice': np.int64(27), 'Beetroot': np.int64(28), 'Bengal Gram Dal (Chana Dal)': np.int64(29), 'Bengal Gram(Gram)(Whole)': np.int64(30), 'Ber(Zizyphus/Borehannu)': np.int64(31), 'Betal

In [ ]:
comm_detail

,Arrival_Date,State,District,Market,Commodity,Variety,Grade,Min_Price,Max_Price,Modal_Price,Commodity_Code,Year,Month,Day
0,2001-01-10,13,144,1166,37,180,0,1380.0,1380.0,1380.0,8,2001,1,10
1,2001-01-10,13,144,1166,130,428,0,1600.0,1900.0,1800.0,9,2001,1,10
2,2001-02-08,16,304,1621,43,982,0,500.0,1060.0,775.0,35,2001,2,8
3,2001-02-08,16,304,1621,138,828,0,1400.0,1700.0,1550.0,75,2001,2,8
4,2001-03-12,21,301,1588,246,828,0,500.0,700.0,600.0,24,2001,3,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9283995,2023-12-31,8,566,3160,120,828,0,8000.0,9000.0,8550.0,103,2023,12,31
9283996,2023-12-31,13,494,2815,12,413,0,30870.0,37209.0,36501.0,140,2023,12,31
9283997,2023-12-31,16,3,2496,120,828,0,3500.0,10000.0,6500.0,103,2023,12,31
9283998,2023-12-31,25,256,1320,212,828,0,4300.0,4600.0,4450.0,12,2023,12,31


In [ ]:
for col in comm_detail.columns:
  print(col, comm_detail[col].dtype)

Arrival_Date datetime64[ns]
State int64
District int64
Market int64
Commodity int64
Variety int64
Grade int64
Min_Price float64
Max_Price float64
Modal_Price float64
Commodity_Code int64
Year int32
Month int32
Day int32


In [ ]:
X = comm_detail.drop(columns=['Arrival_Date', 'Min_Price','Max_Price','Modal_Price','Commodity_Code'], axis=1)
#
Y = comm_detail['Modal_Price']
Z_min=comm_detail['Min_Price']
Z_max=comm_detail['Max_Price']

In [ ]:
X

,State,District,Market,Commodity,Variety,Grade,Year,Month,Day
0,13,144,1166,37,180,0,2001,1,10
1,13,144,1166,130,428,0,2001,1,10
2,16,304,1621,43,982,0,2001,2,8
3,16,304,1621,138,828,0,2001,2,8
4,21,301,1588,246,828,0,2001,3,12
...,...,...,...,...,...,...,...,...,...
9283995,8,566,3160,120,828,0,2023,12,31
9283996,13,494,2815,12,413,0,2023,12,31
9283997,16,3,2496,120,828,0,2023,12,31
9283998,25,256,1320,212,828,0,2023,12,31


In [ ]:
Y

,Modal_Price
0,1380.0
1,1800.0
2,775.0
3,1550.0
4,600.0
...,...
9283995,8550.0
9283996,36501.0
9283997,6500.0
9283998,4450.0


In [ ]:
Z_min

,Min_Price
0,1380.0
1,1600.0
2,500.0
3,1400.0
4,500.0
...,...
9283995,8000.0
9283996,30870.0
9283997,3500.0
9283998,4300.0


In [ ]:
Z_max

,Max_Price
0,1380.0
1,1900.0
2,1060.0
3,1700.0
4,700.0
...,...
9283995,9000.0
9283996,37209.0
9283997,10000.0
9283998,4600.0


In [ ]:
X.columns

Index(['State', 'District', 'Market', 'Commodity', 'Variety', 'Grade', 'Year',
       'Month', 'Day'],
      dtype='object')

In [ ]:
X_c=pd.DataFrame(X, columns=['State', 'District', 'Market', 'Commodity','Variety','Grade', 'Year',
       'Month', 'Day'])
X_c

,State,District,Market,Commodity,Variety,Grade,Year,Month,Day
0,13,144,1166,37,180,0,2001,1,10
1,13,144,1166,130,428,0,2001,1,10
2,16,304,1621,43,982,0,2001,2,8
3,16,304,1621,138,828,0,2001,2,8
4,21,301,1588,246,828,0,2001,3,12
...,...,...,...,...,...,...,...,...,...
9283995,8,566,3160,120,828,0,2023,12,31
9283996,13,494,2815,12,413,0,2023,12,31
9283997,16,3,2496,120,828,0,2023,12,31
9283998,25,256,1320,212,828,0,2023,12,31


In [ ]:
corr_matrix = X_c.corr().abs()
corr_matrix

,State,District,Market,Commodity,Variety,Grade,Year,Month,Day
State,1.000000,0.068497,0.071868,0.034018,0.043259,0.055032,0.137204,0.002352,0.002216
District,0.068497,1.000000,0.406786,0.002886,0.007620,0.012755,0.021443,0.002121,0.001023
Market,0.071868,0.406786,1.000000,0.015147,0.002908,0.013008,0.010421,0.002790,0.000660
Commodity,0.034018,0.002886,0.015147,1.000000,0.258369,0.075033,0.065166,0.001889,0.000907
Variety,0.043259,0.007620,0.002908,0.258369,1.000000,0.015131,0.068228,0.006860,0.004169
Grade,0.055032,0.012755,0.013008,0.075033,0.015131,1.000000,0.011303,0.013603,0.000765
Year,0.137204,0.021443,0.010421,0.065166,0.068228,0.011303,1.000000,0.022795,0.003455
Month,0.002352,0.002121,0.002790,0.001889,0.006860,0.013603,0.022795,1.000000,0.018736
Day,0.002216,0.001023,0.000660,0.000907,0.004169,0.000765,0.003455,0.018736,1.000000


In [ ]:
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
upper

,State,District,Market,Commodity,Variety,Grade,Year,Month,Day
State,NaN,0.068497,0.071868,0.034018,0.043259,0.055032,0.137204,0.002352,0.002216
District,NaN,NaN,0.406786,0.002886,0.007620,0.012755,0.021443,0.002121,0.001023
Market,NaN,NaN,NaN,0.015147,0.002908,0.013008,0.010421,0.002790,0.000660
Commodity,NaN,NaN,NaN,NaN,0.258369,0.075033,0.065166,0.001889,0.000907
Variety,NaN,NaN,NaN,NaN,NaN,0.015131,0.068228,0.006860,0.004169
Grade,NaN,NaN,NaN,NaN,NaN,NaN,0.011303,0.013603,0.000765
Year,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.022795,0.003455
Month,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.018736
Day,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
to_drop = [column for column in upper.columns if any(upper[column] > 0.75)]
len(to_drop), to_drop

(0, [])

In [ ]:
X_columns=list(X.columns)
y_columns=["Modal_Price"]
Z_min_col=["Min_Price"]
Z_max_col=["Max_Price"]
y_columns, X_columns,Z_min_col, Z_max_col

(['Modal_Price'],
 ['State',
  'District',
  'Market',
  'Commodity',
  'Variety',
  'Grade',
  'Year',
  'Month',
  'Day'],
 ['Min_Price'],
 ['Max_Price'])

In [ ]:
correlation_result={}
for i in range(len(X_columns)):
    correlation = X[X_columns[i]].corr(comm_detail["Modal_Price"])
    correlation_result[X_columns[i]]=correlation
correlation_result

{'State': np.float64(-0.03387849279379157),
 'District': np.float64(0.015813298215939187),
 'Market': np.float64(0.022287528827625056),
 'Commodity': np.float64(-0.10735492239133192),
 'Variety': np.float64(-0.007440114346737322),
 'Grade': np.float64(0.0152263503553827),
 'Year': np.float64(0.16192965906346027),
 'Month': np.float64(0.011213663379137822),
 'Day': np.float64(0.002126295171007812)}

In [ ]:
correlation_result=sorted(correlation_result.items(), key = lambda kv:(kv[1], kv[0]),reverse=True)
correlation_result

[('Year', np.float64(0.16192965906346027)),
 ('Market', np.float64(0.022287528827625056)),
 ('District', np.float64(0.015813298215939187)),
 ('Grade', np.float64(0.0152263503553827)),
 ('Month', np.float64(0.011213663379137822)),
 ('Day', np.float64(0.002126295171007812)),
 ('Variety', np.float64(-0.007440114346737322)),
 ('State', np.float64(-0.03387849279379157)),
 ('Commodity', np.float64(-0.10735492239133192))]

In [ ]:
x_train = X[:8500000]
x_test = X[8500000:]
y_train = Y[:8500000]
y_test = Y[8500000:]
z_min_train = Z_min[:8500000]
z_min_test = Z_min[8500000:]
z_max_train = Z_max[:8500000]
z_max_test = Z_max[8500000:]
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((8500000, 9), (783987, 9), (8500000,), (783987,))

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
z_min_train, z_min_test, z_max_train, z_max_test = train_test_split(Z_min, Z_max, test_size=0.2, random_state=42)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((7427189, 9), (1856798, 9), (7427189,), (1856798,))

In [ ]:
scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

In [ ]:
max(x_train.flatten()), min(x_train.flatten()), max(x_test.flatten()), min(x_test.flatten())


(np.float64(4.624834127500788),
 np.float64(-2.6577375299563815),
 np.float64(4.624834127500788),
 np.float64(-2.6577375299563815))

In [ ]:
f = open("/content/drive/MyDrive/models/scaler.pkl", "wb")
pickle.dump(scaler, f)
f.close()

#ML model implementation

##Stage 1


### XGBoost for min price

In [ ]:
import xgboost as xgb

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

In [ ]:
# Stage 1: Predict Min_Price and Max_Price
# features_stage1 = ['State', 'District', 'Market', 'Commodity', 'Variety', 'Grade', 'Year', 'Month', 'Day']
# X_stage1 = comm_detail[features_stage1]
# y_min =comm_detail['Min_Price']
# y_max = comm_detail['Max_Price']

# Train models for Min_Price and Max_Price
model_min = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=1000, learning_rate=0.1, max_depth=10)
model_min.fit(X, Z_min)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=10, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=1000, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [ ]:
y_min_test_pred = model_min.predict(X)
test_r2 = r2_score(Z_min, y_min_test_pred)
print(f"\nTest R2 Score: {test_r2:.4f}")


Test R2 Score: 0.9485


###XGBoost max price

In [ ]:
model_max = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=1000, learning_rate=0.1, max_depth=10)
model_max.fit(X, Z_max)
y_max_test_pred = model_max.predict(X)
test_max_r2 = r2_score(Z_max, y_max_test_pred)
print(f"\nTest R2 Score: {test_max_r2:.4f}")


Test R2 Score: 0.9769


In [ ]:
joblib.dump(model_max, os.path.join('/content/drive/MyDrive/models', 'max_price_xgb_model_10.joblib'))

['/content/drive/MyDrive/models/max_price_xgb_model_10.joblib']

In [ ]:
joblib.dump(model_min, os.path.join('/content/drive/MyDrive/models', 'min_price_xgb_model_10.joblib'))

['/content/drive/MyDrive/models/min_price_xgb_model_10.joblib']

##Stage 2

##LightXGBoost

In [ ]:
import lightgbm as lgb

In [ ]:
comm_detail["Min_Price"] = model_min.predict(X)
comm_detail["Max_Price"] = model_max.predict(X)

In [ ]:
X_stage2 = comm_detail[['State', 'District', 'Market', 'Commodity', 'Variety', 'Grade',
                      'Year', 'Month', 'Day', 'Min_Price', 'Max_Price']]
y_stage2 = comm_detail['Modal_Price']

In [ ]:
train_data = lgb.Dataset(X_stage2, label=y_stage2)

In [ ]:
# Define LightGBM parameters
params = {
    'objective': 'regression',       # For regression tasks
    'metric': 'rmse',               # Evaluation metric
    'max_depth': None,                 # Maximum depth of a tree
    'learning_rate': 0.1,           # Learning rate
    'num_leaves': 31,               # Number of leaves in a tree
    'feature_fraction': 0.8,        # Subsample ratio of columns
    'bagging_fraction': 0.8,        # Subsample ratio of the training instances
    'seed': 42                      # Random seed
}

# Train the model
num_rounds = 1000
lgb_model = lgb.train(params, train_data, num_rounds)

# Make predictions
lgb_y_pred = lgb_model.predict(X_stage2)

# Evaluate the model
# lgb_mae = mean_absolute_error(y_test, lgb_y_pred)
# lgb_mse = mean_squared_error(y_test, lgb_y_pred)
# print(f"LightGBM - MAE: {lgb_mae}, MSE: {lgb_mse}")
lgb_r2 = r2_score(y_stage2,lgb_y_pred)
print(f"LightGBM - R2: {lgb_r2}")

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 1.267489 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1525
[LightGBM] [Info] Number of data points in the train set: 9283987, number of used features: 11
[LightGBM] [Info] Start training from score 2695.921383
LightGBM - R2: 0.9136855245207438


In [ ]:
joblib.dump(lgb_model, os.path.join('/content/drive/MyDrive/models', 'stage2_avg_price_lgb_model_10.joblib'))

['/content/drive/MyDrive/models/stage2_avg_price_lgb_model_10.joblib']